## Décorateurs, Générateurs, Iterateurs, Async/Await en Python et plus encore !

## Décorateurs (@)

> Analogie : C'est comme amballer un cadeau (votre fonction originale) reste le meme, mais vous lui ajoutez du papier cadeau et un ruban (la nouvelle fonctionnalité)

In [ ]:
# Imaginons nous voulions mesurer le temps d"exécution d'une fonction.
import time

def timer_decorateur(func):
    # La fonction "wrapper" enveloppe la fonction originale
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs) # on exécute la fonction originale
        end_time = time.time()
        print(f"{func.__name__} a pris {end_time - start_time} secondes pour s'exécuter.")
        return result
    return wrapper

@timer_decorateur
def longue_operation(nom_tache):
    time.sleep(2)
    print(f"Tache '{nom_tache} terminée !")
    
# on appelle simplement la fonction, le décorateur est déjà appliqué !
longue_operation("Nettoyage de données")
    

Tache 'Nettoyage de données terminée !
longue_operation a pris 2.001107692718506 secondes pour s'exécuter.


In [1]:
# Décorateur de journalisation (logging) : ce décorateur affiche des informations sur une focntion (ses arguments et sa valeur de retour) chaque fois qu'elle est appelée.
def log_function_call(func):
    # Affiche le nom de fonction, ses arguments et sa valeur de retour.
    def wrapper(*args, **kwargs):
        # avant l'appel de la fonction
        print(f"Appel de la fonction '{func.__name__}'...")
        print(f"Arguments positionnels (args): {args}")
        print(f"Arguments nommés (kwargs): {kwargs}")
        
        # Appel de la fonction originale
        result = func(*args, **kwargs)
        
        # Aprés l'appel de la fonction
        print(f"La fonction '{func.__name__}' a retourné : {result}")
        return result
    return wrapper

@log_function_call
def addition(a, b):
    return a+b

@log_function_call
def saluer(nom, message="Bonjour"):
    return f"{message}, {nom} !"

# utilisation
addition(3,5)
saluer("Simone")

Appel de la fonction 'addition'...
Arguments positionnels (args): (3, 5)
Arguments nommés (kwargs): {}
La fonction 'addition' a retourné : 8
Appel de la fonction 'saluer'...
Arguments positionnels (args): ('Simone',)
Arguments nommés (kwargs): {}
La fonction 'saluer' a retourné : Bonjour, Simone !


'Bonjour, Simone !'

## Les Générateurs (yield)


In [3]:
# Fonction classique (gourmande en mémoire pour de grandes listes)
def liste_carres(n):
    resultats = []
    for i in range(n):
        resultats.append(i * i)
    return resultats

# Générateur (très efficace en mémoire)
def generateur_carres(n):
    for i in range(n):
        yield i * i # 'yield' met en pause et "retourne une valeur"
        
# use case
print("Générateur")
# 'gen' est un objet générateur, rien n'a encore été calculé
gen = generateur_carres(10)
print(gen)

# on consommme les valeurs une par une
print(next(gen)) # Affiche 0
print(next(gen)) # Affiche 1
print(next(gen)) # Affiche 4

# on peut aussi l'utiliser directement dans une boucle for
print("Reste des valeurs :")
for carre in gen:
    print(carre) # 9 16 etc....

Générateur
<generator object generateur_carres at 0x00000285DB908110>
0
1
4
Reste des valeurs :
9
16
25
36
49
64
81


> Un générateur est une fonction qui ne retourne pas une valeur unique, mais une séquence de valeurs à la demande, en utilisant le mot-clé yield.
Contrairement à une fonction classique qui retourne un résultat et termine, un générateur se met en pause après chaque yield et reprends là où il s’était arrêté quand on lui demande la valeur suivante.

## Comparaison entre une liste et un générateur

In [ ]:
# Création d'une liste avec 1 million de nombres
grande_liste = liste_carres(1000000) # consomme beaucoup de mémoire (RAM)  IMPORTANT : NE PAS EXECUTER CETTE CELLULE SI VOUS AVEZ PEU DE RAM

Problème :

- La liste occupe beaucoup de mémoire (environ 8 Mo pour 1 million d’entiers).
- Si tu ne traites qu’un élément à la fois, c’est du gaspillage.

In [4]:
# Avec un générateur, on peut traiter les nombres un par un sans tout stocker en mémoire
def generate_nombres(n):
    for i in range(n):
        yield i # 'yield' met en pause et "retourne une valeur"
        
nombre = generate_nombres(1000000) # ici pas de liste en mémoire, juste un générateur créé
for _ in range(10): # on traite les 10 premiers nombres
    print(next(nombre)) # affiche les nombres de 0 à 9

0
1
2
3
4
5
6
7
8
9


Avantages :

- Pas de stockage en mémoire : Les nombres sont générés à la volée.
- Économie de mémoire : Seul le nombre courant est en mémoire.
- Traitement immédiat : Pas besoin d’attendre que tout soit généré.

In [ ]:
#  Traitement "Lazy" (paresseux)
# Les données sont générées uniquement quand on en a besoin.
def lire_gros_fichier(fichier):
    with open(fichier, 'r') as f:
        for ligne in f:  # ⬅️ Générateur implicite !
            yield ligne

for ligne in lire_gros_fichier("huge_file.txt"):
    print(ligne)  # Traite une ligne à la fois

### Exemple Avancé : Pipeline de données avec des génréteurs

> Alors, imaginons que nous ayons une flux de données brutes (ventes, logs, etc...) et nous voulons les nettoyer, les filtrer et les transformer, le tout sans jamais rien stocker en mémoire.

In [5]:
# Nos données sources simulées

donnees_brutes = [
    {"id": 1, "produit": "Pomme", "quantite": 10},
    {"id": 2, "produit": "Orange", "quantite": "cinq"}, # Donnée invalide
    {"id": 3, "produit": "Banane", "quantite": 150},
    {"id": 4, "produit": "Poire", "quantite": 8},
    {"id": 5, "produit": "Fraise", "quantite": 200},
    {"id": 6, "produit": "Raisin", "quantite": 50},
    {"id": 7, "produit": "Melon", "quantite": 12},
    {"id": 8, "produit": "Abricot", "quantite": -5},            # quantite négative (invalide)
    {"id": 9, "produit": "Kiwi", "quantite": "douze"},         # quantite en chaîne (invalide)
    {"id": 10, "produit": "Mangue", "quantite": 30},
    {"id": 11, "produit": "Cerise", "quantite": None},         # valeur manquante (invalide)
    {"id": 12, "produit": "Ananas"},                           # clé quantite manquante (invalide)
    {"id": 13, "produit": "  Poire  ", "quantite": "15"},      # espaces et quantite en chaîne (à normaliser)
]

# Étape 1 : Nettoyage des données, premier générateur
def nettoyer_donnees(donnees):
    # générateur qui nettoie les données
    print("Début du nettoyage des données...")
    for item in donnees:
        try:
            quantite = int(item.get("quantite", 0)) # Convertir en entier, défaut à 0 si manquant
            if quantite < 0:
                continue  # Ignorer les quantités négatives
            item["quantite"] = quantite # Mettre à jour la quantité nettoyée
            item["produit"] = item["produit"].strip().title()  # Normaliser le nom du produit
            yield item # ne produit l'item si il est valide
        except (ValueError, TypeError):
            print(f"Donnée invalide ignorée : {item}")
            continue  # Ignorer les entrées avec des quantités non convertibles
        
# Étape 2 : Filtrage des données, deuxième générateur
def filtrer_donnees(donnees, seuil_quantite):
    print(f"Filtrage des données avec un seuil de quantité > {seuil_quantite}...")
    for item in donnees:
        if item["quantite"] > seuil_quantite:
            yield item
        else:
            print(f"Donnée filtrée (quantité insuffisante) : {item['produit']} avec quantité {item['quantite']}")

# Étape 3 : Transformation des données, troisième générateur
def transformer_donnees(donnees):
    print("Transformation des données...")
    for item in donnees:
        item_transforme = {
            "ID Produit": item["id"],
            "Nom Produit": item["produit"],
            "Quantité en Stock": item["quantite"],
            "Statut": "En Stock" if item["quantite"] > 0 else "Rupture de Stock"
        }
        yield item_transforme
    
# Pipeline complet
pipeline = transformer_donnees(
    filtrer_donnees(
        nettoyer_donnees(donnees_brutes),
        seuil_quantite=10
    )
)

# Consommation du pipeline
for produit in pipeline:
    print(produit)
    

Transformation des données...
Filtrage des données avec un seuil de quantité > 10...
Début du nettoyage des données...
Donnée filtrée (quantité insuffisante) : Pomme avec quantité 10
Donnée invalide ignorée : {'id': 2, 'produit': 'Orange', 'quantite': 'cinq'}
{'ID Produit': 3, 'Nom Produit': 'Banane', 'Quantité en Stock': 150, 'Statut': 'En Stock'}
Donnée filtrée (quantité insuffisante) : Poire avec quantité 8
{'ID Produit': 5, 'Nom Produit': 'Fraise', 'Quantité en Stock': 200, 'Statut': 'En Stock'}
{'ID Produit': 6, 'Nom Produit': 'Raisin', 'Quantité en Stock': 50, 'Statut': 'En Stock'}
{'ID Produit': 7, 'Nom Produit': 'Melon', 'Quantité en Stock': 12, 'Statut': 'En Stock'}
Donnée invalide ignorée : {'id': 9, 'produit': 'Kiwi', 'quantite': 'douze'}
{'ID Produit': 10, 'Nom Produit': 'Mangue', 'Quantité en Stock': 30, 'Statut': 'En Stock'}
Donnée invalide ignorée : {'id': 11, 'produit': 'Cerise', 'quantite': None}
Donnée filtrée (quantité insuffisante) : Ananas avec quantité 0
{'ID Prod

## Generator expression(Expression genératrice)

- List Comprehension (Crée une liste complète en mémoire) : Vous connaissez cette syntaxe qui utilise des crochets [] :
```python
squares = [x**2 for x in range(10)]
```
- Generator Expression (Crée un générateur qui produit des éléments à la demande) : Utilise des parenthèses () au lieu de crochets [] :
```python
squares_gen = (x**2 for x in range(10))
```
Avantages des Generator Expressions :
- Efficacité Mémoire (Lazy Evaluation) : L'expression génératrice ne calcule les valeurs qu'au moment où vous les demandez (par exemple, dans une boucle for). Elle ne stocke jamais toute la collection en mémoire. C'est exactement le même principe que les générateurs créés avec yield.

- Performance : C'est beaucoup plus rapide à démarrer car Python n'a pas besoin d'allouer de la mémoire pour des millions d'éléments d'un coup

## cas d'utilisation par exemple
Utiliser une expression génératrice partout où vous avez besoin de parcourir une séquence une seule fois, sans avoir besoin de la stocker. C'est très courant dans les fonctions d'agrégation.

Exemple : Imaginez que vous vouliez la somme des carrés de 10 millions de nombres.

Mauvaise façon (inefficace) :
```python
# 1. Crée une liste de 10 millions d'éléments en mémoire
# 2. Puis la parcourt pour faire la somme
squares = [x**2 for x in range(10000000)]
total = sum(squares)
```

Bonne façon (efficace) :
```python
'sum()' va "tirer" les valeurs du générateur une par une
# et les additionner sans jamais créer la liste complète.
somme = sum(x * x for x in range(10_000_000))
# (Notez qu'on peut même omettre les parenthèses ()
# quand l'expression est le seul argument d'une fonction)

En résumé : Une expression génératrice est à un générateur ce qu'une "list comprehension" est à une liste. C'est un raccourci syntaxique pour créer des générateurs simples en une seule ligne.

### Exemple d'un generator pipeline

In [4]:
data = [
    "zoo length wrong shinning mathematics shoulder stage example weak roll found evidence species born grain further offer whose stay rope provide everyone ice pound",
    "peace forty almost myself pride roar disappear harbor contain nest expression lost cost happy select jack leader cloth physical farmer rising army every element",
    "onto oxygen thick process until thin feathers personal fruit worse yes eat shot poet feet tales beneath steam thread know you worry best herd",
    "tall serious creature weather close lie grabbed orange past radio chose fact cow public act perfect dirty pot cup was cake cap fix secret",
    "hunter elephant kids against thought becoming source those prevent behind solution suddenly wolf duck long construction thou information closely letter together wet distant show",
    "lonely bow dead effort pleasure arrange dirty torn series what speech individual congress yes solution came daughter throughout business gather come learn cow using",
    "fact thy properly talk statement tropical education serve composed parent unit die door older pull should by almost blue weak found primitive oil pleasure",
    "breath wind unless sister scared sudden selection neighbor direction ground tropical hurt fell us furniture right card shinning guard than paid lion you quietly",
    "round chain act forget trouble magic angry now instance command are queen bowl captain work deep underline vote also foreign service struggle managed complex",
    "favorite mood student mouth wall event settle compound chamber stream floor power season tightly happy require sang radio else ago young rapidly damage clear",
    "hole explain handle early here bush nest cotton does angle clearly wind fruit slightly wrong signal sink needle exact island face married party general",
    "moon frequently complete essential deep complex way dead lot amount lesson grown pride meat knowledge score remove nearly ever tonight automobile decide us won",
    "wood could press wife lay ten blow area dug mental pale glass clothes board six anybody everything apart will other pride ran straight living",
    "vessels excellent traffic pupil choose jungle supply slight trap animal simplest burst same forty fighting away truth sit ill weigh former familiar eaten common",
    "silver case ahead barn whispered kind from breath cave fill forest addition cat manner train children greater accept unusual television but jack fighting went",
    "attack correctly getting tune think joined bent shoulder according roof swept yes managed visit harder particles pole feet yourself knew warn plates ourselves brown",
    "stomach seldom trail pure tool river captured gray farm she equator fight row laid observe flat better look brass cool different why book south",
    "metal church string alphabet nearly number record depth outline remain for where sugar arrange twenty poet struggle body particular after tell mix agree scientific",
    "peace mass forty colony dress it shelter sides printed shout pupil energy law nature also person lake chance drawn noon though crop meant although",
    "dull recent road tone ask exercise almost rice tried log environment part circle enemy seen business parts own stretch problem classroom win done glass",
]


strip_ws = [0] # generator expression removing the spaces from the strings
len_str = [0] # generator expression finding each string's length
less_than = [0] # generator expression all the lengths less than 130

# creating the generator expressions
strip_ws = (s.replace(" ", "") for s in data)
len_str = (len(s) for s in strip_ws)
less_than = (l for l in len_str if l < 130)
# consuming the final generator
# for length in less_than:
#     print(length)

print(list(less_than))
# NOTE the first and second to print out the result of the generator cannot be used at the same time because the first one will have already consumed the generator.
    
# or
# print(list(l for l in (len(s.replace(" ", "")) for s in data) if l < 130))

[118, 114, 128, 118, 120, 123, 128]


Here, we are going to use a generator to clean a wheather API response. 
- Create a function called clean_data that takes a list of dictionaries as input.
- The functions should clean the data so that temps is an interger, the data becomes a DateTime object, the wind_speed is a float. 
- Country is already in lowercase.
- The function return a generator that yields cleaned dictionaries one by one.

In [10]:
from datetime import datetime

data = [{"country": "Somalia","temp": "88","date": "4/5/2046","wind_speed": "23.64mph",},]

def clean_data(data):
    for weather_data in data:
        cleaned_data = {}
        cleaned_data["country"] = weather_data["country"].lower()
        cleaned_data["temp"] = int(weather_data["temp"])
        cleaned_data["date"] = datetime.strptime(weather_data["date"], "%m/%d/%Y")
        cleaned_data["wind_speed"] = float(weather_data["wind_speed"].replace("mph", ""))
        yield cleaned_data
        
# usage
for cleaned in clean_data(data):
    print(cleaned)

{'country': 'somalia', 'temp': 88, 'date': datetime.datetime(2046, 4, 5, 0, 0), 'wind_speed': 23.64}


## Programmation Asynchrone (async/await)

In [2]:
import asyncio
import aiohttp
import time
import pprint # pour une affiche plus "joli" des dictionnaires

# URL de base de l'API publique 
BASE_URL = "https://fakestoreapi.com"

async def fetch_json(session: aiohttp.ClientSession, url: str) -> dict | list | None:
    # Une coroutine utilitaire pour récupérer les données JSON d'une URL et gérer les erreurs de base.
    try:
        async with session.get(url) as response:
            response.raise_for_status() # lève une erreur pour les status 4xx/5xx
            # 'await' ici met cette coroutine en pause, permettant aux autres de s'exécuter
            data = await response.json()
            return data
    except Exception as e:
        print(f"Erreur lirs de la requete vers {url}: {e}")
        return None
    
async def get_categories(session: aiohttp.ClientSession) -> list:
    """
    Étape 1 : Récupérer la liste des noms de catégories.
    """
    print("Étape 1 : Récupération de la liste des catégories...")
    url = f"{BASE_URL}/products/categories"
    categories = await fetch_json(session, url)
    return categories if categories else []

async def get_products_in_category(session: aiohttp.ClientSession, category: str) -> tuple:
    """
    Étape 2 : Récupérer tous les produits pour UNE catégorie donnée.
    Retourne un tuple (nom_categorie, liste_produits)
    """
    print(f"  -> Tâche créée pour la catégorie '{category}'")
    url = f"{BASE_URL}/products/category/{category}"
    products = await fetch_json(session, url)
    return (category, products if products else [])

async def main():
    """
    La coroutine principale qui orchestre le pipeline.
    """
    print("--- Démarrage du pipeline de données asynchrone ---")
    start_time = time.time()
    
    # Créer une seule session pour toutes nos requêtes
    async with aiohttp.ClientSession() as session:
        
        # --- ÉTAPE 1 : Appel unique et bloquant (dans 'main') ---
        # Nous avons besoin des catégories avant de pouvoir continuer,
        # donc nous utilisons 'await' ici.
        categories = await get_categories(session)
        
        if not categories:
            print("Impossible de récupérer les catégories. Arrêt.")
            return

        print(f"\nÉtape 1 terminée. {len(categories)} catégories trouvées : {categories}")
        print("\n--- ÉTAPE 2 : Lancement des tâches en parallèle ---")

        # --- ÉTAPE 2 : Création de la liste des tâches concurrentes ---
        # Nous ne les 'await' PAS encore. Nous créons juste les "promesses".
        tasks = []
        for category_name in categories:
            task = get_products_in_category(session, category_name)
            tasks.append(task)
            
        # --- ÉTAPE 3 : Exécution de toutes les tâches en parallèle ---
        # asyncio.gather() lance toutes les tâches dans 'tasks'
        # et attend qu'elles soient TOUTES terminées.
        # return_exceptions=True empêche le programme de planter si UNE seule requête échoue.
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
    end_time = time.time()
    print("\n--- ÉTAPE 3 : Traitement des résultats ---")

    total_products = 0
    for res in results:
        if isinstance(res, Exception):
            # Si une des tâches a échoué, on l'affiche
            print(f"Échec de la récupération d'une catégorie : {res}")
        else:
            # Sinon, on traite le résultat (category, products)
            category_name, products = res
            print(f"Catégorie '{category_name}': {len(products)} produits trouvés.")
            total_products += len(products)
            
    print("\n--- RÉSUMÉ ---")
    print(f"Nombre total de produits récupérés : {total_products}")
    print(f"Temps total : {end_time - start_time:.2f} secondes")
    
    # Affichons un exemple de données reçues
    if results and not isinstance(results[0], Exception):
        print("\nExemple de données (premier produit de la première catégorie) :")
        pprint.pprint(results[0][1][0])

if __name__ == "__main__":
    # Si c'était synchrone, le temps total serait la somme de tous les appels.
    # En asynchrone, le temps total sera (temps étape 1) + (temps du plus long appel de l'étape 2).
    try:
        asyncio.run(main())
    except RuntimeError:
        # Probablement déjà dans une boucle d'événements (ex: Jupyter Notebook).
        # Tenter d'utiliser nest_asyncio pour permettre l'imbrication des boucles d'événements.
        try:
            import nest_asyncio
            nest_asyncio.apply()
            asyncio.get_event_loop().run_until_complete(main())
        except ImportError:
            print("Le package 'nest_asyncio' est requis pour exécuter asyncio.run() dans ce contexte.")
            print("Installez-le avec : %pip install nest_asyncio")
            raise


--- Démarrage du pipeline de données asynchrone ---
Étape 1 : Récupération de la liste des catégories...

Étape 1 terminée. 4 catégories trouvées : ['electronics', 'jewelery', "men's clothing", "women's clothing"]

--- ÉTAPE 2 : Lancement des tâches en parallèle ---
  -> Tâche créée pour la catégorie 'electronics'
  -> Tâche créée pour la catégorie 'jewelery'
  -> Tâche créée pour la catégorie 'men's clothing'
  -> Tâche créée pour la catégorie 'women's clothing'

--- ÉTAPE 3 : Traitement des résultats ---
Catégorie 'electronics': 6 produits trouvés.
Catégorie 'jewelery': 4 produits trouvés.
Catégorie 'men's clothing': 4 produits trouvés.
Catégorie 'women's clothing': 6 produits trouvés.

--- RÉSUMÉ ---
Nombre total de produits récupérés : 20
Temps total : 0.29 secondes

Exemple de données (premier produit de la première catégorie) :
{'category': 'electronics',
 'description': 'USB 3.0 and USB 2.0 Compatibility Fast data transfers Improve '
                'PC Performance High Capacity

C:\Users\wamba\AppData\Local\Temp\ipykernel_27028\2066890320.py:109: RuntimeWarning: coroutine 'main' was never awaited
  asyncio.get_event_loop().run_until_complete(main())
